In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

2023-06-25 10:16:36.511832: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
(training_images, training_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

In [3]:
# We see that the observations are balanced
np.unique(training_labels, return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000]))

In [4]:
# 60,000 images of 28 x 28 pixels each
training_images.shape

(60000, 28, 28)

In [5]:
# Each image is 28 x 28. Unsure what each value represents, however.
training_images[0].shape

(28, 28)

In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices((training_images, training_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

In [7]:
# We generally want the pixel values to be floating numbers between 0 and 1
train_dataset = train_dataset.map(lambda image, label: (float(image)/255.0, label))
test_dataset = test_dataset.map(lambda image, label: (float(image)/255.0, label))

In [8]:
train_dataset.as_numpy_iterator().next()[0]

2023-06-25 10:16:45.849866: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [60000]
	 [[{{node Placeholder/_1}}]]


array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [9]:
# Clearly understand the concept of batches in machine learning
# This means that the entire dataset is divided into batches of 64 observations
# By sampling 500 of these batches of 64. We are really using only 32,000 observations.
batch_size = 64
train_dataset = train_dataset.batch(batch_size).shuffle(500)
test_dataset = test_dataset.batch(batch_size).shuffle(500)

In [10]:
len(train_dataset.as_numpy_iterator().next()[0])

2023-06-25 10:16:46.086459: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [60000]
	 [[{{node Placeholder/_1}}]]
2023-06-25 10:16:46.087821: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [60000]
	 [[{{node Placeholder/_1}}]]


64

In [24]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape = (28,28)))
model.add(tf.keras.layers.Dense(20, activation = 'relu'))
model.add(tf.keras.layers.Dense(10))

In [25]:
model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate = 0.1),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics = ['accuracy'])

In [26]:
model.fit(train_dataset, epochs = 5)

Epoch 1/5


938/938 [==============================] - 5s 3ms/step - loss: 0.6486 - accuracy: 0.7753
Epoch 2/5
938/938 [==============================] - 4s 3ms/step - loss: 0.4660 - accuracy: 0.8336
Epoch 3/5
938/938 [==============================] - 5s 4ms/step - loss: 0.4280 - accuracy: 0.8468
Epoch 4/5
938/938 [==============================] - 5s 4ms/step - loss: 0.4085 - accuracy: 0.8538
Epoch 5/5
938/938 [==============================] - 5s 4ms/step - loss: 0.3941 - accuracy: 0.8576


In [27]:
model.evaluate(test_dataset)

2023-06-25 10:25:57.459127: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype uint8 and shape [10000,28,28]
	 [[{{node Placeholder/_0}}]]
2023-06-25 10:25:57.459691: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [10000]
	 [[{{node Placeholder/_1}}]]


157/157 [==============================] - 2s 5ms/step - loss: 0.4461 - accuracy: 0.8398


[0.4461049437522888, 0.8398000001907349]

In [28]:
model.save('outputs/model')

INFO:tensorflow:Assets written to: outputs/model/assets


INFO:tensorflow:Assets written to: outputs/model/assets


In [29]:
model = tf.keras.models.load_model('outputs/model')